# Assignment 2

As before, if a question can be answered with 'yes/no', or a numeric value, you may simply state as much. If you incorporate code from the internet (which is not required and generally not advisable), please cite the source within your code (providing a URL is sufficient).

We will go through comparable code and concepts in the live learning sessions. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that no outside searches are required by the assignment!). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Import specific objects
from sklearn.preprocessing import StandardScaler
from ISLP import load_data
from sklearn.neighbors import KNeighborsClassifier
from ISLP import confusion_table


### Question 1: Classification using KNN

We'll now use the `Caravan` dataset from the `ISLP` package. (You may use `Caravan.describe()` to review details of the dataset.) In this dataset, the response variable of interest is `Purchase`, which indicates if a given customer purchased a caravan insurance policy. We will simultaneously use all other variables in the dataset to predict the response variable.

In [5]:
# Load the "Caravan" dataset using the "load_data" function from the ISLP package
Caravan = load_data('Caravan')
Caravan.describe()


MOSTYPE     MAANTHUI      MGEMOMV     MGEMLEEF     MOSHOOFD  \
count  5822.000000  5822.000000  5822.000000  5822.000000  5822.000000   
mean     24.253349     1.110615     2.678805     2.991240     5.773617   
std      12.846706     0.405842     0.789835     0.814589     2.856760   
min       1.000000     1.000000     1.000000     1.000000     1.000000   
25%      10.000000     1.000000     2.000000     2.000000     3.000000   
50%      30.000000     1.000000     3.000000     3.000000     7.000000   
75%      35.000000     1.000000     3.000000     3.000000     8.000000   
max      41.000000    10.000000     5.000000     6.000000    10.000000   

            MGODRK       MGODPR       MGODOV       MGODGE       MRELGE  ...  \
count  5822.000000  5822.000000  5822.000000  5822.000000  5822.000000  ...   
mean      0.696496     4.626932     1.069907     3.258502     6.183442  ...   
std       1.003234     1.715843     1.017503     1.597647     1.909482  ...   
min       0.000000     0.000000     0.000000     0.000000     0.000000  ...   
25%       0.000000     4.000000     0.000000     2.000000     5.000000  ...   
50%       0.000000     5.000000     1.000000     3.000000     6.000000  ...   
75%       1.000000     6.000000     2.000000     4.000000     7.000000  ...   
max       9.000000     9.000000     5.000000     9.000000     9.000000  ...   

            ALEVEN     APERSONG      AGEZONG      AWAOREG       ABRAND  \
count  5822.000000  5822.000000  5822.000000  5822.000000  5822.000000   
mean      0.076606     0.005325     0.006527     0.004638     0.570079   
std       0.377569     0.072782     0.080532     0.077403     0.562058   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.000000     0.000000     0.000000     1.000000   
75%       0.000000     0.000000     0.000000     0.000000     1.000000   
max       8.000000     1.000000     1.000000     2.000000     7.000000   

           AZEILPL     APLEZIER       AFIETS      AINBOED     ABYSTAND  
count  5822.000000  5822.000000  5822.000000  5822.000000  5822.000000  
mean      0.000515     0.006012     0.031776     0.007901     0.014256  
std       0.022696     0.081632     0.210986     0.090463     0.119996  
min       0.000000     0.000000     0.000000     0.000000     0.000000  
25%       0.000000     0.000000     0.000000     0.000000     0.000000  
50%       0.000000     0.000000     0.000000     0.000000     0.000000  
75%       0.000000     0.000000     0.000000     0.000000     0.000000  
max       1.000000     2.000000     3.000000     2.000000     2.000000  

[8 rows x 85 columns]

Before fitting any model, it is essential to understand our data. Answer the following questions about the `Caravan` dataset (Hint: use `print` and `describe`):  
_(i)_ How many observations (rows) does the dataset contain?  

There are 5822 observations(rows) in the dataset.

In [6]:
# print(Caravan.info())
print(Caravan.shape)

(5822, 86)


In [7]:
Caravan.head(4)

MOSTYPE  MAANTHUI  MGEMOMV  MGEMLEEF  MOSHOOFD  MGODRK  MGODPR  MGODOV  \
0       33         1        3         2         8       0       5       1   
1       37         1        2         2         8       1       4       1   
2       37         1        2         2         8       0       4       2   
3        9         1        3         3         3       2       3       2   

   MGODGE  MRELGE  ...  APERSONG  AGEZONG  AWAOREG  ABRAND  AZEILPL  APLEZIER  \
0       3       7  ...         0        0        0       1        0         0   
1       4       6  ...         0        0        0       1        0         0   
2       4       3  ...         0        0        0       1        0         0   
3       4       5  ...         0        0        0       1        0         0   

   AFIETS  AINBOED  ABYSTAND  Purchase  
0       0        0         0        No  
1       0        0         0        No  
2       0        0         0        No  
3       0        0         0        No  

[4 rows x 86 columns]

In [8]:
Caravan.columns

Index(['MOSTYPE', 'MAANTHUI', 'MGEMOMV', 'MGEMLEEF', 'MOSHOOFD', 'MGODRK',
       'MGODPR', 'MGODOV', 'MGODGE', 'MRELGE', 'MRELSA', 'MRELOV', 'MFALLEEN',
       'MFGEKIND', 'MFWEKIND', 'MOPLHOOG', 'MOPLMIDD', 'MOPLLAAG', 'MBERHOOG',
       'MBERZELF', 'MBERBOER', 'MBERMIDD', 'MBERARBG', 'MBERARBO', 'MSKA',
       'MSKB1', 'MSKB2', 'MSKC', 'MSKD', 'MHHUUR', 'MHKOOP', 'MAUT1', 'MAUT2',
       'MAUT0', 'MZFONDS', 'MZPART', 'MINKM30', 'MINK3045', 'MINK4575',
       'MINK7512', 'MINK123M', 'MINKGEM', 'MKOOPKLA', 'PWAPART', 'PWABEDR',
       'PWALAND', 'PPERSAUT', 'PBESAUT', 'PMOTSCO', 'PVRAAUT', 'PAANHANG',
       'PTRACTOR', 'PWERKT', 'PBROM', 'PLEVEN', 'PPERSONG', 'PGEZONG',
       'PWAOREG', 'PBRAND', 'PZEILPL', 'PPLEZIER', 'PFIETS', 'PINBOED',
       'PBYSTAND', 'AWAPART', 'AWABEDR', 'AWALAND', 'APERSAUT', 'ABESAUT',
       'AMOTSCO', 'AVRAAUT', 'AAANHANG', 'ATRACTOR', 'AWERKT', 'ABROM',
       'ALEVEN', 'APERSONG', 'AGEZONG', 'AWAOREG', 'ABRAND', 'AZEILPL',
       'APLEZIER', 'AFIETS',

 
_(ii)_ How many variables (columns) does the dataset contain?  

There are 86 columns in the dataset.


_(iii)_ What 'variable' type is the response variable `Purchase` (e.g., 'character', 'factor', 'numeric', etc)? What are the 'levels' of the variable?    



In [9]:
df = pd.DataFrame(Caravan)
print(df['Purchase'].unique())

['No' 'Yes']


'Purchase' is a 'categorical' response variable that has two categories 'Yes' or 'No'. So there are two 'levels' of the response variable namely 'Yes' and 'No'.


_(iv)_ How many predictor variables do we have (Hint: all variables other than `Purchase`)?  

In [10]:
len(Caravan.columns)

86

There are 86 columns in the dataset. One of the variables 'Purchase' is the response variable. So, there are 85 predictor variables

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the `scaler` method, provided as follows:

In [11]:
# Select predictors (excluding the 86th column)
predictors = Caravan.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    MOSTYPE  MAANTHUI   MGEMOMV  MGEMLEEF  MOSHOOFD    MGODRK    MGODPR  \
0  0.680906  -0.27258  0.406697 -1.216964  0.779405 -0.694311  0.217444   
1  0.992297  -0.27258 -0.859500 -1.216964  0.779405  0.302552 -0.365410   
2  0.992297  -0.27258 -0.859500 -1.216964  0.779405 -0.694311 -0.365410   
3 -1.187437  -0.27258  0.406697  0.010755 -0.970980  1.299414 -0.948264   
4  1.225840  -0.27258  1.672893 -1.216964  1.479559  0.302552 -0.365410   

     MGODOV    MGODGE    MRELGE  ...   ALEVEN  APERSONG   AGEZONG  AWAOREG  \
0 -0.068711 -0.161816  0.427670  ... -0.20291 -0.073165 -0.081055 -0.05992   
1 -0.068711  0.464159 -0.096077  ... -0.20291 -0.073165 -0.081055 -0.05992   
2  0.914172  0.464159 -1.667319  ... -0.20291 -0.073165 -0.081055 -0.05992   
3  0.914172  0.464159 -0.619824  ... -0.20291 -0.073165 -0.081055 -0.05992   
4 -0.068711  0.464159  0.427670  ... -0.20291 -0.073165 -0.081055 -0.05992   

     ABRAND   AZEILPL  APLEZIER   AFIETS   AINBOED  ABYSTAND  
0  0.764971 -0.02

_(v)_ Why is it important to standardize the predictor variables?  


Standardization or Scaling of the data ensures that the features (or predictors) are on a similar scale. It prevents some features dominating over others. Features can have different units or scales. Standardization makes the features comparable. Without standardization, predictors that have larger values or variance can disproportionately influence the model's predictions. Standardization also reduces the impact of outliers. The model becomes less sensitive to extreme values.

Standardization involves removing the mean (mean is subtracted from the value) and dividing the difference by the standard deviation. This makes the mean of each feature equal to 0 and variance of each feature equal to 1.

_(vi)_ Why did we elect not to standard our response variable `Purchase`?  

The predictor variable 'Purchase' is a categorical variable with values 'Yes' and 'No', which is binary or discreet. Its not a continuous numerical value. Discreet binary variables like 'Yes' and ''No' are not standardized.


_(vii)_ A second essential step is to set a random seed. Do so below (Hint: use the `random.seed` function). Why is setting a seed important? Is the particular seed value important? Why or why not?

Setting the seed makes sure that the sequence of random numbers (in this case split of 75% and 25% of the dataset) generates the same split-data each time its run. That way, you can re-produce the same results which is important for testing and debugging purposes. When sharing code, using the seed ensures that others can re-produce the same results. A particular seed value is not important. It can be any integer.

_(viii)_ A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. Extend the code to create a non-overlapping test set for the predictors and response variables.

In [12]:
# setting the seed
np.random.seed(45)

# Create a random vector of True and False values
split = np.random.choice([True, False], size=len(predictors_standardized), replace=True, p=[0.75, 0.25])

# Define the training set for X (predictors)
training_X = predictors_standardized[split]

# Define the training set for Y (response)
training_Y = Caravan.loc[split, 'Purchase']

# Define the testing set for X (predictors)
testing_X = predictors_standardized[~split]

# Define the testing set for Y (response)
testing_Y = Caravan.loc[~split, 'Purchase']

training_X, testing_X = [np.asarray(X) for X in [training_X, testing_X]]


_(ix)_ We are finally set to fit the KNN model. In Python, we can use the `KNeighborsClassifier()` function. Fit the KNN with k=1. (You may review arguments to knn by typing `help(knn.fit)`). 

In [13]:
knn = KNeighborsClassifier(n_neighbors=1) 
knn.fit(training_X, training_Y)
knn_pred_y = knn.predict(testing_X)
confusion_table(knn_pred_y, testing_Y)


c:\Users\hml\.conda\envs\dsi_participant\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\hml\.conda\envs\dsi_participant\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


Truth        No  Yes
Predicted           
No         1305   75
Yes          94    5

Using your fit model, answer the following questions:   
_(x)_ What is the prediction accuracy? (Hint: use the `score` method, and compare your model to `testing_Y`)  


In [14]:
# prediction accuracy
accuracy = knn.score(testing_X, testing_Y)
print("Accuracy calculated using score method", accuracy)

print("Accuracy calculated using confusion matrix", (1305+5)/(1305+5+75+94))

print("Accuracy calculated using mean method", np.mean(knn_pred_y == testing_Y))

Accuracy calculated using score method 0.8857336037863421
Accuracy calculated using confusion matrix 0.8857336037863421
Accuracy calculated using mean method 0.8857336037863421



_(xi)_ What is the predictor error ? (Hint: compute it from the accuracy)

In [15]:
# prediction error rate
error_rate = 1 - accuracy
print("Prediction error rate", error_rate)

Prediction error rate 0.11426639621365786


_(xii)_ How does this prediction error/accuracy compare to what could be achieved via random guesses? To answer this, consider the percent of customers in the `Caravan` dataset who actually purchase insurance, computed below:

In [26]:


# Calculate the percentage of customers who purchase insurance
percentage_purchase = (Caravan['Purchase'].eq('Yes').sum() / len(Caravan['Purchase'])) * 100
print("Yes percentage:", percentage_purchase)

yes_purchase = percentage_purchase/100
no_purchase = 1 - (percentage_purchase/100)
print('Yes: ', yes_purchase)
print('No: ', no_purchase)

random_accuracy = (yes_purchase**2)+(no_purchase**2)
print('Random guess Accuracy:', random_accuracy)


Yes percentage: 5.977327378907591
Yes:  0.05977327378907591
No:  0.9402267262109241
Random guess Accuracy: 0.8875991409407757


For a balanced dataset, where the number of 'Yes' and number of 'No' are the same, the accuracy of the random guess is 50%. The accuracy of the predicted KNN model is 88.57% which is better than 50%.
But, this is a very imbalanced dataset. The number of 'Yes' is 5.98%. The random guess of 50% will not apply to this dataset.

https://towardsdatascience.com/calculating-a-baseline-accuracy-for-a-classification-model-a4b342ceb88f

The KNN model accuracy of 88.57% is very close to the weighted random-guess accuracy of 88.76% which takes the imbalance of the dataset into account. Although the KNN model with one neighbour is relevant, it is worth trying to model with neighbours more than one.

_(xiii)_ Fit a second KNN model, with $K=3$. Does this model perform better (i.e., have higher accuracy, compared to a random guess)?

In [17]:
knn = KNeighborsClassifier(n_neighbors=3) 
knn.fit(training_X, training_Y)
knn_pred_y = knn.predict(testing_X)

print(confusion_table(knn_pred_y, testing_Y))
print("Accuracy", np.mean(knn_pred_y == testing_Y))

Truth        No  Yes
Predicted           
No         1373   76
Yes          26    4
Accuracy 0.9310344827586207


The accuracy of 93.10% with K=3 is better than 88.57% with K=1. It is also better than the random accuracy of 88.76%.

# Criteria

|Criteria            |Complete           |Incomplete          |
|--------------------|---------------|--------------|
|Classification using KNN|All steps are done correctly and the answers are correct.|At least one step is done incorrectly leading to a wrong answer.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/applying_statistical_concepts/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
